<a href="https://colab.research.google.com/github/Near-o7/challenge2-data-science-LATAM/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

Se extraen los datos desde la URL proporcionada (en formato JSON) y se almacenan en un DataFrame

In [20]:
import pandas as pd

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Se observa que algunas columnas contienen registros anidados en forma de diccionarios, por lo que es necesario normalizar los datos. Este proceso se abordará en la sección '🔧 Transformación'.

#🔧 Transformación

## Identificando el tipo de datos

In [21]:
df.info()
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


,0
customerID,object
Churn,object
customer,object
phone,object
internet,object
account,object


Se observa que las el tipo de datos de las columnas se representa como objeto. Luego de normalizar correctamente los datos este aspecto deberá ser abordado.

## Diccionario de datos

A continuación se presenta un diccionario de datos del conjunto de datos

- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente

## Identificando columnas

A priori, podemos identificar algunas columnas clave para nuestro análisis, basándonos en el análisis exploratorio inicial y el diccionario de datos. Entre las más relevantes se encuentran:

- **Churn**: Indica si un cliente ha abandonado la empresa o no.
- **PaymentMethod**: Podría ayudarnos a detectar si existe una relación entre el método de pago y la propensión a la evasión.
- **Tenure**: Refleja la antigüedad de los clientes, lo que podría ser útil para entender su lealtad.
- **Charges.Monthly y Charges.Total**: Permiten evaluar si existe una relación entre el monto cobrado a los clientes y la probabilidad de evasión.

Existen otras columnas de interés que analizaremos con mayor detalle en las siguientes secciones.


## Transformando y limpiando datos

In [29]:
df_customer = pd.json_normalize(df['customer'])
df_phone = pd.json_normalize(df['phone'])
df_internet = pd.json_normalize(df['internet'])
df_account_normalized = pd.json_normalize(df['account'])

df_final = pd.concat([
    df[['customerID', 'Churn']],
    df_customer,
    df_phone,
    df_internet,
    df_account_normalized
], axis=1)

df_final.head()



,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


In [38]:
categorical_cols = [
    'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
    'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling',
    'PaymentMethod', 'Churn'
]

for col in categorical_cols:
    if col in df_final.columns:
        print(f"Valores únicos en '{col}': {df_final[col].unique()}")
        print(f"Conteo de valores en '{col}':\n{df_final[col].value_counts()}\n")
        print(f"Total de valores en '{col}': {df_final[col].value_counts().sum()}\n")

Valores únicos en 'gender': ['Female' 'Male']
Conteo de valores en 'gender':
gender
Male      3675
Female    3592
Name: count, dtype: int64

Total de valores en 'gender': 7267

Valores únicos en 'Partner': ['Yes' 'No']
Conteo de valores en 'Partner':
Partner
No     3749
Yes    3518
Name: count, dtype: int64

Total de valores en 'Partner': 7267

Valores únicos en 'Dependents': ['Yes' 'No']
Conteo de valores en 'Dependents':
Dependents
No     5086
Yes    2181
Name: count, dtype: int64

Total de valores en 'Dependents': 7267

Valores únicos en 'PhoneService': ['Yes' 'No']
Conteo de valores en 'PhoneService':
PhoneService
Yes    6560
No      707
Name: count, dtype: int64

Total de valores en 'PhoneService': 7267

Valores únicos en 'MultipleLines': ['No' 'Yes' 'No phone service']
Conteo de valores en 'MultipleLines':
MultipleLines
No                  3495
Yes                 3065
No phone service     707
Name: count, dtype: int64

Total de valores en 'MultipleLines': 7267

Valores únicos en

Se observa que la columna Churn contiene valores vacíos. De un total de 7,267 registros, 224 presentan valores faltantes, lo que representa aproximadamente un 3%. En consecuencia, se procederá a eliminar estas filas con valores nulos en Churn del DataFrame.

In [31]:
print("\nVerificando valores nulos:")
print(df_final.isnull().sum()) # Conteo de nulos por columna
print(df_final.isnull().sum() / len(df_final) * 100) # Porcentaje de nulos


Verificando valores nulos:
customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges.Monthly     0
Charges.Total       0
dtype: int64
customerID          0.0
Churn               0.0
gender              0.0
SeniorCitizen       0.0
Partner             0.0
Dependents          0.0
tenure              0.0
PhoneService        0.0
MultipleLines       0.0
InternetService     0.0
OnlineSecurity      0.0
OnlineBackup        0.0
DeviceProtection    0.0
TechSupport         0.0
StreamingTV         0.0
StreamingMovies     0.0
Contract            0.0
PaperlessBilling    0.0
PaymentMethod       0.0
Charges.Monthly     0.0
Charges.Total    

In [34]:
# Contar el número de filas en el DataFrame
total_rows = df_final.shape[0]
print(f"Número total de filas en df_final: {total_rows}")

# Contar el número de customerID únicos
unique_customer_ids = df_final['customerID'].nunique()
print(f"Número de customerID únicos: {unique_customer_ids}")

# Verificar si hay duplicados
if total_rows > unique_customer_ids:
    print("\n¡ADVERTENCIA: Hay customerID duplicados en el DataFrame!")
    # Opcional: Mostrar los customerID que están duplicados y cuántas veces aparecen
    duplicated_ids = df_final['customerID'][df_final['customerID'].duplicated(keep=False)]
    print("\nEjemplos de customerID duplicados y su conteo:")
    print(duplicated_ids.value_counts().head()) # Muestra los 5 más frecuentes

else:
    print("\nTodos los customerID son únicos. ¡No hay duplicados en esta columna!")

Número total de filas en df_final: 7267
Número de customerID únicos: 7267

Todos los customerID son únicos. ¡No hay duplicados en esta columna!


#📊 Carga y análisis

#📄Informe final